In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '../')
from ESINet.forward import create_forward_model
from ESINet.simulations import run_simulations, create_eeg
from ESINet.util import *
from ESINet.ann import *
from ESINet.viz import parula

pth_fwd = 'forward_models/ico3/'

In [ ]:
create_forward_model(pth_fwd, sampling='ico3')

## Simulate some source activity and EEG data

In [ ]:
settings = dict(n_simulations=10000,
                n_sources=(1, 5),   # Specify the range of number of present sources 
                extents=(1, 4),     # Specify the range of size of the sources (in neighborhood orders)
                amplitudes=(1, 100), # Specify the range of amplitudes (in nAm)
                shape='both',       # Shape can be 'flat', 'gaussian' or 'both'
                durOfTrial=0)       # ESINet works with single-instance data, i.e duration of the trial segments must be 0 here

# Calculate random brain activity base on settings:
sources_sim = run_simulations(pth_fwd, **settings)
# Calculate the corresponding EEG signals using the forward model and some noise
eeg_sim = create_eeg(sources_sim, pth_fwd, snr=2, n_trials=20)

## Plot a simulated sample and the corresponding EEG

In [ ]:
%matplotlib qt
from ESINet.viz import parula
sample = np.random.choice(np.arange(1000))
title = f'Simulation {sample}'
# Topographic plot
eeg_sim[sample].average().plot_topomap(0.5, cmap=parula)
# Source plot
sources_sim.plot(hemi='both', initial_time=sample/100, surface='white', colormap=parula, title=title, time_viewer=False, backend='pyvista')

## Load and train ConvDip with simulated data

In [ ]:
# Find out input and output dimensions based on the shape of the leadfield 
input_dim, output_dim = load_leadfield(pth_fwd).shape
# Initialize the artificial neural network model.
model = get_model(input_dim, output_dim, n_layers=1, n_neurons=128)
# Train the model
model, history = train_model(model, sources_sim, eeg_sim, delta=1, )

## Evaluate ConvDip
Let's evaluate our model!

In [ ]:
# Load some files from the forward model
leadfield = load_leadfield(pth_fwd)
info = load_info(pth_fwd)

In [ ]:
import time
%matplotlib qt

# Simulate a brand new sample:
sources_eval = run_simulations(pth_fwd, **settings)
eeg_eval = create_eeg(sources_eval, pth_fwd)

# Calculate the ERP (average across trials):
eeg_sample = np.squeeze( eeg_eval )

# Predict
start = time.time()
source_predicted = predict(model, eeg_sample, pth_fwd)
end = time.time()

# Visualize ground truth...
title = f'Ground Truth'
xx = sources_eval.plot(hemi='both', initial_time=0.5, surface='white', colormap=parula, title=title, time_viewer=False, backend='pyvista')

# ... and prediction
title = f'ConvDip Prediction'
yy = source_predicted.plot(hemi='both', initial_time=0.5, surface='white', colormap=parula, title=title, time_viewer=False, backend='pyvista')

# ... and the 'True' EEG topography
title = f'Simulated EEG'
eeg_eval[0].average().plot_topomap([0], title=title, cmap=parula)

print(f'\nInverse solution and pre- & postprocessing were calculated within {(end-start)*1000:.1f} ms')